# Homework problem

In an office building there are 11 floors on which 30 tenants are to be located. Each tenant has given space requirements that may vary with the floor to which he is assigned. If tenant j is assigned to floor i, he needs $a_{ij}$ $m^2$ of floor space Floor i is assumed to have an area of $b_i$ $m^2$. Suppose that each $m^2$ of space on floor i that is rented yields $c_i$ $/month for the company. To maximize the revenue which tenant should be assigned to which floor?

### Discrete Value Representation

You may use discrete value representation like the following:

| 1 | 2 | 3 | ... | 30 |
|---|---|---|-----|----|
| 4 | 5 | 1 | ... | 5  |



In [1]:
import numpy as np
from itertools import product

In [2]:
def generate_testcase(num_floors, num_tenants, verbose=True):
    needs = np.random.randint(10, size=(num_floors, num_tenants))
    capacities = np.random.randint(30, size=(num_floors))
    unit_rents = np.random.randint(10, size=(num_floors))

    if verbose:
        print('needs:', needs)
        print('capacities:', capacities)
        print('unit_rents:', unit_rents)
    return needs, capacities, unit_rents

In [3]:
def generate_all_placements(num_floors, num_tenants):
    return product(*[range(num_floors) for x in range(num_tenants)])

In [4]:
def get_score(placement, needs, unit_rents, capacities):
    num_floors = unit_rents.shape[0]
    used_space = np.zeros(num_floors)

    for tenant, floor in enumerate(placement):
        used_space[floor] += needs[floor, tenant]

    if np.any(used_space > capacities):
        return -1
    return unit_rents.dot(used_space)

In [41]:
def greedy(needs, capacities, unit_rents, verbose=True):
    def _get_score(placement):
        return get_score(
            placement=placement,
            needs=needs,
            unit_rents=unit_rents,
            capacities=capacities)

    (num_floors, num_tenants) = needs.shape
    permutations = list(generate_all_placements(num_floors, num_tenants))

    if verbose:
        print(f'there are a total of {len(permutations)} permutations')

    solutions = list(zip(permutations, map(_get_score, permutations)))
    solutions = sorted(solutions, key=lambda x: -x[1])
    if verbose:
        print(f'best 5 solutions: {solutions[:5]}')
        print(f'finished greedy algorithm at {solutions[0]}')

    return solutions[0]

In [37]:
def change_one(max_value):
    def operator(current):
        neighbors = []
        for i in range(len(current)):
            for v in range(max_value):
                if current[i] is v:
                    next
                new = list(current)
                new[i] = v
                neighbors.append(new)

        return neighbors

    return operator

In [38]:
def local_search(needs,
                 capacities,
                 unit_rents,
                 operator,
                 search_size=10,
                 verbose=True):
    def _get_score(placement):
        return get_score(
            placement=placement,
            needs=needs,
            unit_rents=unit_rents,
            capacities=capacities)

    def select_best(neighbors):
        scores = list(zip(neighbors, map(_get_score, neighbors)))
        scores = sorted(scores, key=lambda x: -x[1])
        return scores[0]

    (num_floors, num_tenants) = needs.shape

    solutions = np.random.choice(
        range(num_floors), size=(search_size, num_tenants))
    solution, score = select_best(solutions)
    if verbose:
        print(f'started local search from {solution} with score:{score}')

    while True:
        neighbors = operator(solution)
        cand, cand_score = select_best(neighbors)
        if cand_score <= score:
            break

        print(f'{cand} has better score: {cand_score}')
        solution, score = cand, cand_score
    print(f'local optimum at {solution} => {score}')
    return solution, score

In [56]:
NUM_FLOORS = 10
NUM_TENANTS = 5
needs, capacities, unit_rents = generate_testcase(
    num_floors=NUM_FLOORS, num_tenants=NUM_TENANTS, verbose=True)

needs: [[7 9 4 1 8]
 [6 2 9 4 3]
 [9 9 8 6 0]
 [5 8 4 0 8]
 [1 9 2 8 5]
 [9 9 9 0 4]
 [5 1 5 0 7]
 [1 1 4 6 3]
 [7 2 2 9 5]
 [5 1 1 3 3]]
capacities: [13 21 25  3 20 25  5 15  6 13]
unit_rents: [8 5 0 6 5 3 8 9 2 0]


In [53]:
greedy_solution, greedy_score = greedy(
    needs=needs, capacities=capacities, unit_rents=unit_rents, verbose=True)

there are a total of 100000 permutations
best 5 solutions: [((3, 9, 2, 1, 9), 209.0), ((3, 9, 1, 3, 9), 206.0), ((3, 9, 2, 3, 9), 206.0), ((3, 9, 3, 1, 9), 206.0), ((3, 9, 3, 3, 9), 203.0)]
finished greedy algorithm at ((3, 9, 2, 1, 9), 209.0)


In [54]:
operator = change_one(max_value=NUM_FLOORS)
local_solution, local_score = local_search(
    needs=needs,
    capacities=capacities,
    unit_rents=unit_rents,
    operator=operator)

started local search from [4 9 2 4 7] with score:96.0
[3, 9, 2, 4, 7] has better score: 148.0
[3, 9, 2, 4, 9] has better score: 190.0
[3, 9, 2, 1, 9] has better score: 209.0
local optimum at [3, 9, 2, 1, 9] => 209.0


In [55]:
if (local_score < greedy_score):
    print(
        f'local optima performed worse: performance {local_score*100/greedy_score:0.0f}%'
    )
else:
    print('local search rocks')

local search rocks


In [46]:
def hard_task():
    NUM_FLOORS = 30
    NUM_TENANTS = 20
    needs, capacities, unit_rents = generate_testcase(
        num_floors=NUM_FLOORS, num_tenants=NUM_TENANTS, verbose=True)

    operator = change_one(max_value=NUM_FLOORS)

    local_solution, local_score = local_search(
        needs=needs,
        capacities=capacities,
        unit_rents=unit_rents,
        operator=operator,
        search_size=100)


hard_task()

needs: [[8 2 8 8 8 0 4 1 5 0 7 7 9 5 0 4 1 4 3 2]
 [3 0 0 0 5 3 0 5 6 7 5 9 6 3 6 8 4 1 7 4]
 [5 9 2 7 7 1 4 2 1 8 7 9 4 7 2 6 1 6 8 6]
 [3 0 7 5 5 2 1 8 5 6 4 4 2 8 3 6 9 0 6 9]
 [7 1 1 9 2 6 0 1 8 7 2 6 1 0 3 7 3 3 2 0]
 [7 2 4 0 2 8 8 1 5 5 9 6 9 1 3 6 8 8 6 7]
 [8 8 2 4 6 8 9 5 2 6 0 2 3 7 0 4 9 7 1 1]
 [2 2 3 2 1 7 6 7 2 5 3 7 6 2 0 0 2 8 1 1]
 [9 0 9 7 2 8 3 0 0 6 0 0 6 4 6 2 4 3 5 6]
 [2 3 8 7 0 2 6 5 2 3 9 4 1 1 2 5 3 2 7 3]
 [7 2 0 3 5 0 9 2 6 8 3 0 8 6 0 5 2 2 1 8]
 [9 1 8 4 2 9 1 6 8 6 5 8 6 3 8 2 0 5 0 6]
 [6 2 0 4 2 0 7 9 1 3 5 4 6 2 0 9 2 4 6 7]
 [7 1 5 9 7 6 9 2 4 4 4 6 1 5 0 3 5 5 2 8]
 [5 3 0 6 3 3 1 1 9 7 2 5 3 5 7 8 9 2 4 9]
 [9 0 0 1 6 5 6 8 8 5 8 6 6 6 8 8 5 1 5 6]
 [8 2 3 1 5 6 1 8 8 2 8 0 8 1 2 8 9 4 2 9]
 [6 5 0 6 9 3 9 7 4 3 3 4 4 1 5 7 1 7 3 8]
 [7 7 2 5 8 6 3 9 1 0 9 5 5 3 0 6 0 9 1 8]
 [5 8 3 4 3 1 1 1 6 0 6 6 0 5 3 0 4 8 8 0]
 [5 8 2 0 9 2 6 9 3 0 8 7 0 4 3 7 4 4 7 9]
 [5 9 8 3 4 4 9 0 9 3 9 2 9 5 6 8 5 3 0 4]
 [4 6 2 5 1 0 6 5 9 1 2 5 1 7 5 2 1 6 0 6]
 [9 